# 1.1. Flask_Login

### **`flask_login`**

- 사용자 로그인 시, 사용자의 session 정보를 **HTTP Response** 에 담아 전달
- 이를 기반으로 Flask 에서 사용자 식별 가능

### **`동작 원리`**

- 사용자 로그인 시, 로그인 정보를 User Class 에서 객체로 가져오고, **LoginManager()** 에 추가하여 **Session** 생성
    - Flask 서버가 리턴 시에 **Session** 정보를 웹페이지에 전달
<br>
<br>
- current_user 객체에 해당 객체 저장
    - current_user.id: 사용자 ID (unicode)
    - cuurrent_user.is_authenticated: 사용자 로그인 여부 (Boolean)
    - attr 추가 가능
<br>
<br>
- 로그인 후 웹페이지로 Flask 서버 접근 시, 전달받은 **Session** 의 정보를 기반으로 접근
    - **Session** 정보에서 id 추출, **LoginManager()** 에서 다루는 id 일 경우, **@login_required** Decorator 추가된 API 접근 허용
<br>
<br>
- 사용자 로그아웃 시, **LoginManager()** 에서 id 제거

### **`User Class`** 구현

- **User Class** 는 **flask_login** 라이브러리의 **UserMixin** 클래스를 상속받아 구현이 가능
    - **is_authenticated**: This property should return True if the user is authenticated
    - **is_active**: This property should return True if this is an active user (계정 중지된 사용자 확인)
    - **is_anonymous**: This property should return True if this is an anonymous user
    - **get_id()**: This method must return a unicode that uniquely identifies this user, and can be used to load the user from the user_loader callback
<br>
<br>
- 필요 시, Override 가능

# 1.2. Flask_login Code

### **`Initialization`**

- **Session Generation** 을 위해 **Flask app** 에서 **Secret Key** 를 정의

```python
app.secret_key = os.urandom(24)
```

- **flask_login** 라이브러리에서 **Session** 관리
    - 아래 코드에서 "strong" 적용 시, 세션 보안 강화

```python
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.session_protection = "strong"
```

### **`Session 생성`**

- **User Class** 를 기반으로 사용자 객체 생성 후, **flask_login.login_user()** 에 사용자 객체 삽입하면 **해당 사용자 Session** 생성
<br>
<br>
- 만들어진 **Session** 은 **HTTP Response** 에 담겨 사용자의 웹브라우저에 전달
    - **HTTP Response** 의 **Set-Cookie** 헤더에 **Session** 담아 전달
    - 사용자의 웹브라우저에서는 이후 해당 서버로 **HTTP Request** 전송 시 **Session** 정보를 자동으로 담아 전송
    - **flask_login** 은 **HTTP Request** 에서 **Session** 정보 추출 후 사용자 식별
<br>
<br>
- **Session** 생성 시, **HTTP Request** 의 **IP** 와 **user agent** 등의 정보를 함께 참조하여 **Session** 생성
    - **Session** 이 탈취당하더라도, 해킹이 어려움
    - 동일한 **Session** 으로 요청을 보내더라도 **HTTP Response** 는 실제 사용자 **IP** 로 전달되어 보안에 용이

```python
from flask_login import login_user

user = User.create(user_email, blog_id)
login_user(user)
```

### **`사전 선언 함수`**

- 로그인 후 최초 **current_user** 호출 시
    - 사용자 정보를 **flask_login** 과 함께 연동된 **User Class** 를 통해 가져옴

```python
@login_manager.user_loader
def load_user(user_id):
    return User.get(user_id)
```

- **@login_required** Decorator 로 로그인 후, 페이지 접근 제어 가능
    - 로그인이 안된 상태로 특정 페이지 접근 시, **@login_manager.unauthorized_handler** 에 정의된 함수를 호출

```python
@login_manager.unauthorized_handler
def unauthorized():
    return make_response(jsonify(success=False), 401)
```

### **`User Class 구현`**

- **User Class** 는 **UserMixin Class** 를 상속
- **attribute** 로 사용자를 식별할 수 있는 id 를 가지고 있어야 한다.

```python
class User(UserMixin):
    def __init__(self, user_id, user_email, blog_id):
        self.id = user_id
        self.user_email = user_email
        self.blog_id = blog_id
        
    def get_id(self):
        return str(self.id)
    
    @staticmethod
    def get(user_id):
        mysql_db = conn_mysqldb()
        db_cursor = mysql_db.cursor()
        sql = "SELECT * FROM user_info WHERE USER_ID = '" + \
            str(user_id) + "'"
        db_cursor.execute(sql)
        user = db_cursor.fetchone()
        if not user:
            db_cursor.close()
            return None
        
        print(user)
        user = User(user_id=user[0], user_email=user[1], blog_id=user[2])
        db_cursor.close()
        return user
    
    @staticmethod
    def find(user_email):
        mysql_db = conn_mysqldb()
        db_cursor = mysql_db.cursor()
        sql = "SELECT * FROM user_info WHERE USER_EMAIL = '" + \
            str(user_email) + "'"
        db_cursor.execute(sql)
        user = db_cursor.fetchone()
        if not user:
            db_cursor.close()
            return None
        
        print(user)
        user = User(user_id=user[0], user_email[1], blog_id=user[2])
        db_cursor.close()
        return user
    
    @staticmethod
    def create(user_email, blog_id):
        user = User.find(user_email)
        if user == None:
            mysql_db = conn_mysqldb()
            db_cursor = mysql_db.cursor()
            sql = "INSERT INTO user_info (USER_EMAIL, BLOG_ID) VALUES ('%s', '%s')" % (
                str(user_email), str(blog_id))
            db_cursor.execute(sql)
            mysql_db.commit()
            return User.find(user_email)
        else:
            return user
```